In [1]:
import sys, os, csv
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
import rdkit.Chem.rdFMCS as MCS

In [2]:
import py3Dmol
from ipywidgets import interact, interactive, fixed

def drawit(m,p,confId=-1):
        mb = Chem.MolToMolBlock(m,confId=confId)
        p.removeAllModels()
        p.addModel(mb,'sdf')
        p.setStyle({'stick':{}})
        p.setBackgroundColor('0xeeeeee')
        p.zoomTo()
        return p.show()

In [3]:
#load UAA smile strings
smi_db = {}
lines = open("UAA-info.csv", 'r').readlines()
for l in lines[:]:
    es = l.split(' ')
    ndx = es[0][:-1]
    smi = es[1].strip()
    #print ndx, smi
    smi_db[ndx] = smi
smi_db

{'A-K': 'N[C@H](C(O)=O)CCCCNC(OCC1=CC=CC=C1[N+]([O-])=O)=O',
 'A-Y': 'N[C@H](C(O)=O)CC1=CC=C(OCC2=C([N+]([O-])=O)C=CC=C2)C=C1',
 'B-K': 'N[C@H](C(O)=O)CCCCNC(OCC1=CC2=C(OCO2)C=C1[N+]([O-])=O)=O',
 'B-Kax': 'N[C@H](C(O)=O)CCCCNC(O[C@H](C)C1=CC2=C(OCO2)C=C1[N+]([O-])=O)=O',
 'B-Kep': 'N[C@H](C(O)=O)CCCCNC(O[C@@H](C)C1=CC2=C(OCO2)C=C1[N+]([O-])=O)=O',
 'B-Y': 'N[C@H](C(O)=O)CC1=CC=C(OCC2=CC(OCO3)=C3C=C2[N+]([O-])=O)C=C1',
 'B-Yax': 'N[C@H](C(O)=O)CC1=CC=C(O[C@H](C)C2=CC(OCO3)=C3C=C2[N+]([O-])=O)C=C1',
 'B-Yep': 'N[C@H](C(O)=O)CC1=CC=C(O[C@@H](C)C2=CC(OCO3)=C3C=C2[N+]([O-])=O)C=C1',
 'C-Kax': 'N[C@H](C(O)=O)CCCCNC(O[C@H]1CCCCC/C=C/1)=O',
 'C-Kep': 'N[C@H](C(O)=O)CCCCNC(O[C@@H]1CCCCC/C=C/1)=O',
 'C-Yax': 'N[C@H](C(O)=O)CC1=CC=C(O[C@H]2CCCCC/C=C/2)C=C1',
 'C-Yep': 'N[C@H](C(O)=O)CC1=CC=C(O[C@@H]2CCCCC/C=C/2)C=C1'}

In [4]:
#define reaction
rxn_smarts = "[C:1](=[O:2])[OH:3].[OH:4][PH0:5][OH:6]>>[C:1](=[O:2])[OH0:4][P:5][O-:6]"
rxn = AllChem.ReactionFromSmarts(rxn_smarts)

In [5]:
#load AMP template
suppl = Chem.SDMolSupplier("AMP.sdf")
for mol in suppl:
    AMP_tmpl = mol
suppl = Chem.SDMolSupplier("stub-Y.sdf")
for mol in suppl:
    stubY = mol
suppl = Chem.SDMolSupplier("stub-K.sdf")
for mol in suppl:
    stubK = mol

In [6]:
pv = py3Dmol.view(width=400,height=400)
interact(drawit, m=fixed(stubK),p=fixed(pv),confId=(0,AMP_tmpl.GetNumConformers()-1));

A Jupyter Widget

In [7]:
pv = py3Dmol.view(width=400,height=400)
interact(drawit, m=fixed(stubY),p=fixed(pv),confId=(0,AMP_tmpl.GetNumConformers()-1));

A Jupyter Widget

In [8]:
import numpy
def CalcConfRMS(N, conf1, conf2):
    ssr = 0                                                                        
    for i in range(N):                                             
        d = conf1.GetAtomPosition(i).Distance(conf2.GetAtomPosition(i))              
        ssr += d * d                                                                 
    ssr /= N
    return numpy.sqrt(ssr)

In [9]:
def GenMutiConfMol(mol, tmpl, num_iter, rms_cut, w):
    #constrained and align
    res = MCS.FindMCS([mol, tmpl], threshold=0.9, completeRingsOnly=True)
    p = Chem.MolFromSmarts(res.smartsString)
    core = AllChem.DeleteSubstructs(AllChem.ReplaceSidechains(tmpl,p), Chem.MolFromSmiles('*'))
    core.UpdatePropertyCache()
    print Chem.MolToSmiles(core)
    #generate confs
    conformers = None
    last_iter = 0
    for i in xrange(num_iter):
        AllChem.ConstrainedEmbed(mol, core, randomseed=i+111)
        ebrms = float(mol.GetProp('EmbedRMS'))
        if ebrms<0.10:
            #check 
            if conformers == None:
                #save the first one
                conformers = Chem.MolFromMolBlock( Chem.MolToMolBlock(mol) )
                Chem.SanitizeMol(conformers)
                conformers = Chem.AddHs(conformers)
                conformers.AddConformer(mol.GetConformer(0))
                w.write(mol)
            else:
                confs = conformers.GetConformers()
                rmslst = [ CalcConfRMS(mol.GetNumAtoms(), conf, mol.GetConformer(0)) for conf in confs ]
                minrms = min(rmslst)
                if minrms > rms_cut:
                    #save if not close any old rotamer
                    conformers.AddConformer(mol.GetConformer(0))
                    w.write(mol)
                    print i, 'RMS:', minrms
                    last_iter = i
                else:
                    if i-last_iter>100:
                        print "Warning: early quit at iter=", i
                        break
    return conformers

In [13]:
for ndx in smi_db.keys():
    k = ndx
    t = ndx[2]
    smi = smi_db[k]
    #init
    d = "U"+str(k)
    if not os.path.isdir(d):
        os.mkdir(d)
    #create
    print k, smi
    UAA = AllChem.MolFromSmiles(smi) #Chem.MolToSmiles(mol, isomericSmiles=True)
    #react
    products = rxn.RunReactants((UAA, AMP_tmpl))
    print len(products)
    writer = Chem.SDWriter(d+"/confs.sdf")
    for prod in products[1:]:
        #fix prod
        product = prod[0]
        Chem.SanitizeMol(product)
        #product = Chem.RemoveHs(product)
        product = Chem.AddHs(product)
        
        if t == "K": stub = stubK
        if t == "Y": stub = stubY
        GenMutiConfMol(product, stub, 5000, 0.5, writer)
    writer.close()

B-Kax N[C@H](C(O)=O)CCCCNC(O[C@H](C)C1=CC2=C(OCO2)C=C1[N+]([O-])=O)=O
2
Nc1ncnc2c1ncn2C1OC(COP(=O)([O-])OC(=O)C(N)CCCCNC=O)C(O)C1O
5 RMS: 35.33575290372326
6 RMS: 3.8278243308125397
7 RMS: 3.2639624251667128
8 RMS: 0.8592190833934815
9 RMS: 1.8804220566584753
10 RMS: 0.7790676546108033
11 RMS: 2.754141487236613
13 RMS: 1.382820313181712
15 RMS: 0.7233056574176979
20 RMS: 1.8541485073086732
21 RMS: 0.6139362082105388
24 RMS: 0.8940659495186041
25 RMS: 0.8957581865996923
29 RMS: 0.7234109550918755
31 RMS: 2.791395361630957
32 RMS: 1.1854270419971769
36 RMS: 0.5953332321477789
38 RMS: 1.6872855430165798
39 RMS: 0.8224985045143258
43 RMS: 0.7777752401849435
47 RMS: 0.715912786573336
50 RMS: 1.8442831384969902
52 RMS: 0.5948588503282357
54 RMS: 0.6184059382557243
58 RMS: 1.647775389528959
60 RMS: 1.7516631659318131
64 RMS: 0.7650319039663838
65 RMS: 0.730807163725872
66 RMS: 0.6528709763303042
67 RMS: 0.7629853259686816
69 RMS: 0.6632030834605014
76 RMS: 0.6318634004054127
78 RMS: 0.6302865

809 RMS: 0.5592392603786727
817 RMS: 0.647160718420113
823 RMS: 0.5095691427890577
825 RMS: 0.5338084841411189
827 RMS: 0.545781960843728
836 RMS: 0.5132362574546505
842 RMS: 0.5973786133920349
844 RMS: 0.5790374187954023
846 RMS: 0.5044518424665926
850 RMS: 0.629980785145643
853 RMS: 0.6771863885254202
857 RMS: 0.6265675219697556
858 RMS: 0.6048404392188723
860 RMS: 0.5134410208364344
863 RMS: 0.6304515037092594
865 RMS: 0.6200251195036494
873 RMS: 0.5216659154994778
882 RMS: 0.5256088252239182
885 RMS: 0.5417607349879708
889 RMS: 0.597871219529661
892 RMS: 0.5935637791660551
894 RMS: 0.6172653073752615
899 RMS: 0.6453400263940885
901 RMS: 0.550542077025463
907 RMS: 0.5106721567094181
909 RMS: 0.5423966627552791
912 RMS: 0.5560920743051005
913 RMS: 0.5691868654384891
914 RMS: 0.616335381586522
917 RMS: 0.5053679643744843
925 RMS: 0.5500551324593081
928 RMS: 0.5216538032283521
931 RMS: 0.5505735842341847
936 RMS: 0.6119800813923034
937 RMS: 0.6061326104570698
939 RMS: 0.617364393585769

1987 RMS: 0.6101082094102434
1999 RMS: 0.5426719112188052
2000 RMS: 0.5696721602890756
2009 RMS: 0.5585081935371875
2012 RMS: 0.5572478943538569
2016 RMS: 0.5259745702387284
2024 RMS: 0.5348899599244226
2025 RMS: 0.5138652624059351
2028 RMS: 0.5053374418042729
2029 RMS: 0.5105814421507359
2031 RMS: 0.5461248983597995
2053 RMS: 0.5056708530249844
2056 RMS: 0.6293920912966032
2058 RMS: 0.5150733631882661
2059 RMS: 0.5395234287666074
2063 RMS: 0.5830859105924624
2064 RMS: 0.5898956810292775
2065 RMS: 0.5556159236685247
2066 RMS: 0.5215140931665978
2067 RMS: 0.5283077432161917
2073 RMS: 0.5478510883195057
2074 RMS: 0.5280384188696452
2076 RMS: 0.6362969452778449
2080 RMS: 0.5400514133582712
2088 RMS: 0.5274368388222669
2089 RMS: 0.6046738960166215
2094 RMS: 0.5432773437754592
2100 RMS: 0.5217550664178532
2101 RMS: 0.5228734823802775
2102 RMS: 0.5097612684892152
2103 RMS: 0.5142250685311734
2107 RMS: 0.571661053717311
2108 RMS: 0.5506828540658788
2110 RMS: 0.5767544903445113
2122 RMS: 0.586

3463 RMS: 0.5785183208947512
3464 RMS: 0.5654084767478391
3476 RMS: 0.5508190731811035
3483 RMS: 0.5515820890069832
3492 RMS: 0.5175478242833671
3496 RMS: 0.5171460315534044
3497 RMS: 0.5971724235709062
3498 RMS: 0.5925431827272257
3500 RMS: 0.5301967711365645
3502 RMS: 0.5312333780552684
3507 RMS: 0.527534353936579
3511 RMS: 0.5025928309822952
3519 RMS: 0.5264214520997205
3520 RMS: 0.5250438421381225
3522 RMS: 0.8126724071801149
3530 RMS: 0.7152471544845662
3534 RMS: 0.5097381748446235
3543 RMS: 0.5187521621387691
3547 RMS: 0.5370711251284511
3555 RMS: 0.5391578957288653
3563 RMS: 0.5683341128556039
3567 RMS: 0.5749811678367213
3568 RMS: 0.5551482324233938
3578 RMS: 0.558834928777576
3586 RMS: 0.5246663603244249
3588 RMS: 0.5100302243879083
3589 RMS: 0.520449691381221
3590 RMS: 0.5218929607705732
3595 RMS: 0.5621102201334008
3603 RMS: 0.5112104984542255
3607 RMS: 0.5048501160999754
3619 RMS: 0.5109419899739271
3623 RMS: 0.5191310463756645
3630 RMS: 0.5118525524974071
3639 RMS: 0.68259

111 RMS: 0.6554492072269548
112 RMS: 0.8806575408133271
114 RMS: 0.8316618426723786
117 RMS: 0.7269162651453415
118 RMS: 0.711429805603781
119 RMS: 0.8254905309571618
120 RMS: 0.7288006212278246
121 RMS: 0.9077204274428943
124 RMS: 0.5906760578402154
126 RMS: 0.7458681752273514
128 RMS: 0.6970869113064916
129 RMS: 0.9927474839926619
131 RMS: 0.6497724660623116
132 RMS: 0.7134452832350927
135 RMS: 0.8246658199326142
138 RMS: 0.7791536944712035
139 RMS: 0.7617612586639729
140 RMS: 0.5852277457114335
141 RMS: 0.5157108713691207
142 RMS: 0.6062084076665262
143 RMS: 0.6310494605188615
145 RMS: 0.7976322220332372
148 RMS: 0.761030913553101
150 RMS: 0.6521723568415216
151 RMS: 0.6819167117763671
160 RMS: 0.7000033971475184
162 RMS: 0.8124986586786342
163 RMS: 0.9771570141607874
164 RMS: 0.8633924120327874
165 RMS: 0.6493844659126445
167 RMS: 1.314986026712719
169 RMS: 0.8140082474601119
170 RMS: 0.7224763620207914
175 RMS: 0.822053320277827
176 RMS: 0.5853087453721615
177 RMS: 0.7359597361425

...

1807 RMS: 0.529295036852602
1819 RMS: 0.6392401031149665
1836 RMS: 0.5228450574347306
1846 RMS: 0.6581347246081813
1851 RMS: 0.6230896218373441
1857 RMS: 0.5105951289146345
1860 RMS: 0.5171232132680496
1865 RMS: 0.5030576602037178
1867 RMS: 0.5276144720059499
1873 RMS: 0.5253554437157598
1875 RMS: 0.5277221677725743
1876 RMS: 0.5334674006213114
1892 RMS: 0.6224831275740241
1893 RMS: 0.5049000060060858
1895 RMS: 0.511210310906409
1899 RMS: 0.6385021409737861
1904 RMS: 0.5813939160008016
1925 RMS: 0.5396343722881625
1930 RMS: 0.5070340770945427
1932 RMS: 0.5294190181348462
1935 RMS: 0.5254768713940061
1937 RMS: 0.5170666421077869
1938 RMS: 0.5142821751436804
1973 RMS: 0.528053470165622
1974 RMS: 0.5465080620903451
1976 RMS: 0.6269345226280119
1984 RMS: 0.5002190010199663
1993 RMS: 0.5652323331384005
1999 RMS: 0.5980766152601524
2002 RMS: 0.5187108487328147
2006 RMS: 0.5891177216854955
2023 RMS: 0.5520859415754241
2027 RMS: 0.5480932523753501
2031 RMS: 0.6177061035375564
2043 RMS: 0.57962

4298 RMS: 0.5043511547019021
4316 RMS: 0.5699552605332081
4323 RMS: 0.5204382971476955
4327 RMS: 0.5292755194918901
4328 RMS: 0.5123350862895935
4347 RMS: 0.5126787693884085
4354 RMS: 0.5512948157569482
4357 RMS: 0.5077712639143338
4371 RMS: 0.5361166201637475
4372 RMS: 0.5049216442758186
4375 RMS: 0.5050411816045105
4406 RMS: 0.521779327301129
4427 RMS: 0.5596708746557674
4438 RMS: 0.5327355716290773
4444 RMS: 0.5007804329938895
4451 RMS: 0.5153536304981893
4453 RMS: 0.5344900706085448
4465 RMS: 0.564399901313259
4496 RMS: 0.5192171537630478
4518 RMS: 0.5299422540953107
4525 RMS: 0.5053729738664245
4568 RMS: 0.5174476716064209
4572 RMS: 0.5203139691348274
4584 RMS: 0.5508645812024827
4586 RMS: 0.5133010440078364
4604 RMS: 0.5626965974585258
4611 RMS: 0.5132926824394746
4617 RMS: 0.549324201465936
4624 RMS: 0.5845808920541155
4647 RMS: 0.5666150367723616
4651 RMS: 0.5330328097273495
4654 RMS: 0.500844256154383
4657 RMS: 0.501463773961924
4659 RMS: 0.5067278860725933
4679 RMS: 0.5042101

309 RMS: 0.5895984271970066
310 RMS: 0.5532812911115353
311 RMS: 0.5800154459851605
325 RMS: 0.5183834844423757
331 RMS: 0.6354569964455143
332 RMS: 0.6376110466116661
337 RMS: 0.558485446960981
345 RMS: 0.7626936024436808
346 RMS: 0.6027694194457139
349 RMS: 0.5927764965686703
351 RMS: 0.5317606930299571
355 RMS: 0.7123093437618447
356 RMS: 0.7332063597488826
359 RMS: 0.555584685074202
360 RMS: 0.5969970713354352
367 RMS: 0.5961041167481927
369 RMS: 0.6035599270679537
370 RMS: 0.5739556987376236
375 RMS: 0.5779906354641313
376 RMS: 0.59382376082382
377 RMS: 0.5594836290801847
380 RMS: 0.5875246868466157
387 RMS: 1.7865470331065012
390 RMS: 0.5301204508556263
392 RMS: 0.5988683887438365
393 RMS: 0.546961416017397
396 RMS: 0.6643305875906262
399 RMS: 0.5086110195725431
403 RMS: 0.5088171931646134
409 RMS: 0.5342310793086152
411 RMS: 0.5777034373629983
414 RMS: 0.5706881533654026
421 RMS: 0.5877424307961454
429 RMS: 0.5373242543453677
438 RMS: 0.5992843397884879
440 RMS: 0.66213328946760

3203 RMS: 0.522119650810849
3209 RMS: 0.5180890372476298
3237 RMS: 0.5403993363590569
3265 RMS: 0.5363976443628997
3267 RMS: 0.5251910005827759
3269 RMS: 0.525098500604984
3320 RMS: 0.5017644597399499
3361 RMS: 0.5597234361522803
3371 RMS: 0.5023508814808988
3376 RMS: 0.689785746737225
3399 RMS: 0.5026264484953508
3498 RMS: 0.5202390441808444
3501 RMS: 0.548166621631502
3508 RMS: 0.5163187737361002
3513 RMS: 0.5564968890067105
3580 RMS: 0.514610827899026
3586 RMS: 0.5698487514459191
3609 RMS: 0.5344415541485995
3628 RMS: 0.5447275297190203
3672 RMS: 0.5341186886618736
3700 RMS: 0.5779004129096585
3720 RMS: 0.534040125204088
3724 RMS: 0.5038123279904813
3738 RMS: 0.5210912915733182
3767 RMS: 0.5226231319702511
3783 RMS: 0.5205229051950855
3831 RMS: 0.5325043545513864
3847 RMS: 0.5316305423413744
3936 RMS: 0.5657503348130566
3949 RMS: 0.5200412219161977
3984 RMS: 0.5239990899564296
3987 RMS: 0.7368234218391123
3990 RMS: 0.5294277003116334
3991 RMS: 0.6028371286082843
3995 RMS: 0.50749088

2299 RMS: 0.5580939394171781
2324 RMS: 0.5322658385158443
2373 RMS: 0.5013009653424297
2376 RMS: 0.5256143754510544
2378 RMS: 0.5005076891828103
2390 RMS: 0.5353231356938362
2394 RMS: 0.5002340396126937
2399 RMS: 0.5033646379524707
2426 RMS: 0.5530796151382141
2474 RMS: 0.6632647502862815
2500 RMS: 0.5019736993153815
2549 RMS: 0.5362276334541929
2553 RMS: 0.5282355206200637
2560 RMS: 0.535947562433997
2586 RMS: 0.5774499403614911
2612 RMS: 0.5015120973246504
2617 RMS: 0.5036278093131348
2637 RMS: 0.569493010505235
2639 RMS: 0.5753603577924957
2640 RMS: 0.5039240949034431
2684 RMS: 0.5017431186313208
2710 RMS: 0.5083406866699852
2713 RMS: 0.6764659755681901
2748 RMS: 0.5160854676666577
2759 RMS: 0.5368829353055927
2780 RMS: 0.6169549989835715
2781 RMS: 0.5195485896674421
2804 RMS: 0.6252444365881401
2890 RMS: 0.5052664554276508
2899 RMS: 0.5032161517902549
2979 RMS: 0.618186512452501
3043 RMS: 1.3045852981511568
3052 RMS: 0.502723630112139
3065 RMS: 0.5864924951124536
3109 RMS: 0.577157